# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [34]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [35]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [36]:
spaceship.dropna(inplace=True)

spaceship['Cabin'].unique()

spaceship['Cabin'] = spaceship['Cabin'].str[0]

spaceship = spaceship.drop(columns=['PassengerId', 'Name'])

non_numeric_cols = spaceship.select_dtypes(include=['object']).columns
spaceship = pd.get_dummies(spaceship, columns=non_numeric_cols, drop_first=True)


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [37]:
#your code here
features = spaceship.select_dtypes(include = np.number)
target = spaceship['Transported']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state = 0)

In [39]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [40]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [41]:
#your code here
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth = 20), n_estimators = 100, max_samples = 1000)

In [42]:
bagging_reg.fit(X_train_norm, y_train)

,estimator,DecisionTreeR...(max_depth=20)
,n_estimators,100
,max_samples,1000
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,None
,verbose,0


In [43]:
BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20), max_samples=1000, n_estimators=100)

,estimator,DecisionTreeR...(max_depth=20)
,n_estimators,100
,max_samples,1000
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,None
,verbose,0


- Evaluate your model

In [44]:
#your code here
pred = bagging_reg.predict(X_test_norm)

print("R2 score", bagging_reg.score(X_test_norm, y_test))

R2 score 0.35528651652030496


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [45]:
#your code here
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

- Run Grid Search

In [49]:
bag_reg = BaggingRegressor(DecisionTreeRegressor())

In [50]:
model = GridSearchCV(estimator = bag_reg, param_grid = grid, cv=5)

In [51]:
model.fit(X_train_norm, y_train)

,estimator,BaggingRegres...eeRegressor())
,param_grid,"{'estimator__max_depth': [10, 30, ...], 'estimator__max_leaf_nodes': [250, 500, ...], 'n_estimators': [50, 100, ...]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [52]:
GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [10, 30, 50],
                         'estimator__max_leaf_nodes': [15, 30, 45],
                         'n_estimators': [50, 100, 200, 500]})

,estimator,BaggingRegres...eeRegressor())
,param_grid,"{'estimator__max_depth': [10, 30, ...], 'estimator__max_leaf_nodes': [15, 30, ...], 'n_estimators': [50, 100, ...]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [53]:
model.best_params_

{'estimator__max_depth': 10,
 'estimator__max_leaf_nodes': 250,
 'n_estimators': 200}

In [54]:
best_model = model.best_estimator_

- Evaluate your model

In [57]:
pred = model.predict(X_test_norm)

print("R2 score", model.score(X_test_norm, y_test))

R2 score 0.3583859264581566
